<a href="https://colab.research.google.com/github/poonamtaj/2022_DSPG_Sahel/blob/main/DSPG_Sahel_Niger_zonalStat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

In [ ]:
import ee
import geemap
import os
import pandas as pd
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Ncnaju47lDk7eZjNe0sP7C5RwFZKeYyTlwyIhhKMd3U&tc=DSYuIQFkWJNs7cFUYLsUqaiTEMpj6bZZ1helctt4Y14&cc=c6psl6Vaq3pNo399rVA8bp94Y7lRq6N_8I3YHqLLf5I

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj_d-9HXPJCYR3Ewsq1HBjkyXrw5Viw-oSCPJD4C07HppIya-JN33g

Successfully saved authorization token.


In [ ]:
Map = geemap.Map()
Map

In [ ]:
## add your shapefile Admin 2 Departments (total 67)
niger_L2_shp = ee.FeatureCollection('projects/poonamtaj-geemap/assets/niger_admin2') ##change the asset id for the relevant shapefile
Map.centerObject(niger_L2_shp, 5)
Map.addLayer(niger_L2_shp, {}, 'Niger Level 2 Departments')

In [ ]:
## add your shapefile Admin 3 Communs (total 266)
niger_L3_shp = ee.FeatureCollection('projects/poonamtaj-geemap/assets/niger_admin3') ##change the asset id or the relevant shapefile
Map.centerObject(niger_L3_shp, 5)
Map.addLayer(niger_L3_shp, {}, 'Niger Level 3 Communes')

# CHIRPS (Pentad) Time series zonal statistics at admin 2 level

In [ ]:
## Time series on multiple polygons using CHIRPS dataset 
## https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD
image_collection_name = "UCSB-CHG/CHIRPS/PENTAD"
date_start = '1981-01-01'
date_end = '2022-04-30'
name = 'CHIRPS'

image_collection = (
    ee.ImageCollection(image_collection_name)
    .filter(ee.Filter.date(date_start, date_end)).select('precipitation')
)

# For visualization purposes
image = image_collection.first()
image = image.clip(niger_L3_shp)

palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

vizParams = {
    'min': 0, 
    'max': 1080,
    'palette': palette, 
    'opacity': 0.6
}

Map.addLayer(image, vizParams, 'CHIRPS Rainfall')
Map

In [ ]:
# Multi-layered image that will give us time series data
precipitaion_ts = image_collection.toBands()

In [ ]:
## extract the zonal statistics for our admin units as csv file, aggregation with 'SUM'
outputSum = '/rainfallSum_ts_niger_adm2.csv'
geemap.zonal_statistics(precipitaion_ts, niger_L2_shp, outputSum, statistics_type='SUM', scale=5000) ## replace with niger_L3_shp

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /rainfall_ts_niger.csv


In [ ]:
df5 = pd.read_csv(outputSum)
df5

In [ ]:
## extract the zonal statistics for our admin units asa  csv file, aggregation with 'MEAN'
outputMean = '/rainfallMean_ts_niger_adm2.csv'
geemap.zonal_statistics(precipitaion_ts, niger_L2_shp, outputMean, statistics_type='MEAN', scale=5000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /rainfallMean_ts_niger_adm3.csv


In [ ]:
df6 = pd.read_csv(outputMean)
df6

In [ ]:
## extract the zonal statistics for our admin units as a csv file, aggregation with 'MEDIAN'
outputMedian = '/rainfallMedian_ts_niger_adm2.csv'
geemap.zonal_statistics(precipitaion_ts, niger_L2_shp, outputMedian, statistics_type='MEDIAN', scale=5000) ## replace with niger_L3_shp

In [ ]:
df6 = pd.read_csv(outputMedian)
df6

# NOAA NDVI Time Series zonal statistics at admin 2 level



In [ ]:
## Time series on multiple polygons using NOAA- AVHRR NDVI dataset 
## https://developers.google.com/earth-engine/datasets/catalog/NOAA_CDR_AVHRR_NDVI_V5

## we may have to split the time period and extract data in two periods.
## two periods: (1) '1982-01-01' to '2000-12-31' (2) '2001-01-01' to '2021-12-31'

image_collection_nameNOAA = "NOAA/CDR/AVHRR/NDVI/V5"
date_start = '1982-01-01' 
date_end = '2000-12-31'
name = 'NDVI'

image_collection_NOAA = (
    ee.ImageCollection(image_collection_nameNOAA)
    .filter(ee.Filter.date(date_start, date_end)).select('NDVI')
)

# For visualization purposes
image_NOAA = image_collection_NOAA.first()
image_NOAA = image_NOAA.clip(niger_L2_shp)

palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']

vizParams = {
    'min': -9998, 
    'max': 9998,
    'palette': palette, 
    'opacity': 0.6
}

Map.addLayer(image_NOAA, vizParams, 'NOAA NDVI')
Map

Map(bottom=3993.0, center=[17.416680667959515, 9.347053286942527], controls=(WidgetControl(options=['position'…

In [ ]:
# Multi-layered image that will give us time series data
NOAA_ts = image_collection_NOAA.toBands()

In [ ]:
## extract the zonal statistics for our admin units as a csv file, aggregation with 'MEAN'
outputNDVIMean = '/ndviMean_ts_niger_adm2.csv'
geemap.zonal_statistics(NOAA_ts, niger_L2_shp, outputNDVIMean, statistics_type='MEAN', scale=5600)

In [ ]:
df7 = pd.read_csv(outputNDVIMean)
df7

In [ ]:
## extract the zonal statistics for our admin units as a csv file, aggregation with 'MEDIAN'
outputNDVIMedian = '/ndviMedian_ts_niger_adm2.csv'
geemap.zonal_statistics(NOAA_ts, niger_L2_shp, outputNDVIMedian, statistics_type='MEDIAN', scale=5600)

In [ ]:
df7 = pd.read_csv(outputNDVIMedian)
df7